In [2]:
!pip install requests
!pip install selenium

In [3]:
import base64
import requests
import datetime
from urllib.parse import urlencode
from urllib.parse import urlparse
import webbrowser
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import config
import json

In [4]:
client_id = config.client_id
client_secret = config.client_secret

In [71]:
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_did_expire = True
    client_id = None
    client_secret = None
    did_perform_grant_flow_auth = False
    authorize_url = "https://accounts.spotify.com/authorize"
    redirected_url = None
    scope = None
    headers = None
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def perform_auth_grant_flow(self):
        client_id = self.client_id
        authorize_url = self.authorize_url
        redirect_uri = "http://localhost:8888/callback"
        authorize_url = self.authorize_url
        scope = self.scope
        if scope is None:
            raise Exception("lack of scope, set permissions scope")
        
        parameters = urlencode({"client_id": client_id, "redirect_uri": redirect_uri, "response_type": "token", 
                                "scope": scope})
        request = requests.get(f"{authorize_url}?{parameters}")
        if request.status_code in range(200,299):
            #webbrowser.open(f"{authorize_url}?{parameters}")
            driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
            driver.get(f"{authorize_url}?{parameters}")
            try:
                WebDriverWait(driver, 60).until(EC.url_contains("access"))
            finally:
                url = driver.current_url
                driver.quit();
                return url
        else:
            raise Exception("failed to request")
    
    def set_redirected_url(self, url):
        self.redirected_url = url
        
    def get_redirected_url(self):
        return self.redirected_url
    
    def parse_redirected_url(self, url):
        if 'error' in url:
            raise Exception("user denied access")
            
        
        spotify.set_redirected_url(url)
        query = urlparse(spotify.get_redirected_url())[5]
        parts = query.split('&')
    
        for i in range(len(parts)):
            parts[i] = parts[i].split('=')[1]
        return parts
    
    def set_access_token_and_expire(self, parts):
        self.access_token = parts[0]
        expires_in_seconds = int(parts[2])
        current_time = datetime.datetime.now()
        expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
        self.token_expires = expires
        self.access_token_did_expire = expires < current_time
    
    def get_access_token(self):
        return self.access_token
    
    def get_expires(self):
        return self.token_expires
    
    ### different scopes must be space separated
    def set_scope(self, scope_string):
        self.scope = scope_string
        
    def get_scope(self):
        return self.scope
    
    def set_headers(self, access_token):
        self.headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
        
    def get_headers(self):
        return self.headers
    
    def get_current_user_playlists(self):
        
        if "playlist-read-private" not in self.scope:
            raise Exception("didn't find playlist-read-private in scope")
        
        version = "v1"
        endpoint = f"https://api.spotify.com/{version}/me/playlists"
        headers = self.headers
        
        if headers is None:
            raise Exception("didn't set headers")
            
        r = requests.get(endpoint, headers=headers)
        
        if r.status_code == 204:
            raise Exception("No content in request")
        
        return r.json()
        

In [72]:
spotify = SpotifyAPI(client_id, client_secret);
spotify.set_scope("playlist-read-private")
url = spotify.perform_auth_grant_flow()

In [73]:
query = spotify.parse_redirected_url(url)
spotify.set_access_token_and_expire(query)
spotify.set_headers(spotify.get_access_token())

In [74]:
content = spotify.get_current_user_playlists();
print(content)

# print(r.json()['item']['artists'][0]['id'])

# print("===")

# artist_id = r.json()['item']['artists'][0]['id']

# endpoint_artist = "https://api.spotify.com/v1/artists/"
# artist_url = f"{endpoint_artist}{artist_id}"

# r = requests.get(artist_url, headers=headers)
# print(r.json()['genres'])

{'href': 'https://api.spotify.com/v1/users/kuba007x/playlists?offset=0&limit=20', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2UA39JwqwiRSXHtfuGbGpk'}, 'href': 'https://api.spotify.com/v1/playlists/2UA39JwqwiRSXHtfuGbGpk', 'id': '2UA39JwqwiRSXHtfuGbGpk', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d0000b2730e33698e02004a608a15c177ab67616d0000b273937111e160868a084f3441eaab67616d0000b2739576057551af1b221acb1eb8ab67616d0000b273aeabbf974c351ca213e85abd', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d0000b2730e33698e02004a608a15c177ab67616d0000b273937111e160868a084f3441eaab67616d0000b2739576057551af1b221acb1eb8ab67616d0000b273aeabbf974c351ca213e85abd', 'width': 300}, {'height': 60, 'url': 'https://mosaic.scdn.co/60/ab67616d0000b2730e33698e02004a608a15c177ab67616d0000b273937111e160868a084f3441eaab67616d0000b2739576057551af1b221acb1eb8ab67616d0000b273aeabbf974c351ca21

In [62]:
print(spotify.get_expires())

2021-03-24 17:57:21.880699
